In [21]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf


In [3]:
calls = pd.read_csv('data/calls_for_service.csv', warn_bad_lines=True)
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA


Decoding day of the week using `merge`

In [4]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
df_dow = pd.DataFrame(dow)

calls = pd.merge(calls.drop(columns="Day", errors="ignore"), 
         df_dow, left_on='CVDOW', right_index=True).sort_index()
calls

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...
5222,19092686,BURGLARY AUTO,12/02/2019 12:00:00 AM,08:30,BURGLARY - VEHICLE,1,09/10/2020 07:00:11 AM,"1000 GILMAN ST\nBerkeley, CA\n(37.88014, -122....",1000 GILMAN ST,Berkeley,CA,Monday
5223,19043965,THEFT MISD. (UNDER $950),08/05/2019 12:00:00 AM,23:30,LARCENY,1,09/10/2020 07:00:08 AM,"2300 WARD ST\nBerkeley, CA\n(37.860105, -122.2...",2300 WARD ST,Berkeley,CA,Monday
5224,19052923,BURGLARY AUTO,09/19/2019 12:00:00 AM,13:30,BURGLARY - VEHICLE,4,09/10/2020 07:00:09 AM,"1500 SHATTUCK AV\nBerkeley, CA\n(37.880227, -1...",1500 SHATTUCK AV,Berkeley,CA,Thursday
5225,19049410,DISTURBANCE,08/05/2019 12:00:00 AM,10:00,DISORDERLY CONDUCT,1,09/10/2020 07:00:09 AM,"2900 REGENT ST\nBerkeley, CA\n(37.857787, -122...",2900 REGENT ST,Berkeley,CA,Monday


Extract latitude and longitude by regex.

In [5]:
calls_lat_lon = (
    calls['Block_Location'].str.replace("\n", "\t") 
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
)
calls_lat_lon.head(10)

,Lat,Lon
0,NaN,NaN
1,NaN,NaN
2,37.868574,-122.270415
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,37.871883,-122.301255


Calculate the fraction of null values in latitude and longitude.

In [6]:
(~calls_lat_lon.isnull()).mean()

Lat    0.963076
Lon    0.963076
dtype: float64

In [7]:
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
calls.merge(calls_lat_lon, left_index=True, right_index=True)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,NaN,NaN
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,NaN,NaN
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday,37.868574,-122.270415
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday,NaN,NaN
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222,19092686,BURGLARY AUTO,12/02/2019 12:00:00 AM,08:30,BURGLARY - VEHICLE,1,09/10/2020 07:00:11 AM,"1000 GILMAN ST\nBerkeley, CA\n(37.88014, -122....",1000 GILMAN ST,Berkeley,CA,Monday,37.88014,-122.297498
5223,19043965,THEFT MISD. (UNDER $950),08/05/2019 12:00:00 AM,23:30,LARCENY,1,09/10/2020 07:00:08 AM,"2300 WARD ST\nBerkeley, CA\n(37.860105, -122.2...",2300 WARD ST,Berkeley,CA,Monday,37.860105,-122.261901
5224,19052923,BURGLARY AUTO,09/19/2019 12:00:00 AM,13:30,BURGLARY - VEHICLE,4,09/10/2020 07:00:09 AM,"1500 SHATTUCK AV\nBerkeley, CA\n(37.880227, -1...",1500 SHATTUCK AV,Berkeley,CA,Thursday,37.880227,-122.26936
5225,19049410,DISTURBANCE,08/05/2019 12:00:00 AM,10:00,DISORDERLY CONDUCT,1,09/10/2020 07:00:09 AM,"2900 REGENT ST\nBerkeley, CA\n(37.857787, -122...",2900 REGENT ST,Berkeley,CA,Monday,37.857787,-122.257013


Translate date time

In [9]:
calls["EVENTDT"][0]

'12/09/2019 12:00:00 AM'

In [10]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

Timestamp('2019-12-09 00:00:00')

In [11]:
# Verify the translations
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

,transformed,original
0,2019-12-09,12/09/2019 12:00:00 AM
1,2019-08-18,08/18/2019 12:00:00 AM
2,2019-10-23,10/23/2019 12:00:00 AM
3,2019-12-01,12/01/2019 12:00:00 AM
4,2019-08-10,08/10/2019 12:00:00 AM


In [12]:
# extract time field
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

0    13:00:00
1    17:20:00
2    10:45:00
3    18:40:00
4    22:51:00
Name: EVENTTM, dtype: object

Use built-in python datetime combine function to combine the correct date and correct time field

In [13]:
from datetime import datetime
timestamps = pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1)
timestamps.head()

0   2019-12-09 13:00:00
1   2019-08-18 17:20:00
2   2019-10-23 10:45:00
3   2019-12-01 18:40:00
4   2019-08-10 22:51:00
dtype: datetime64[ns]

In [14]:
calls['timestamp'] = timestamps
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,timestamp
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,2019-12-09 13:00:00
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,2019-08-18 17:20:00
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday,2019-10-23 10:45:00
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday,2019-12-01 18:40:00
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday,2019-08-10 22:51:00


Time range

In [15]:
calls['timestamp'].min()
calls['timestamp'].max()

Timestamp('2019-12-20 01:30:00')

Visualizations for datetime

In [22]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow].iplot(kind='bar', yTitle='Count')

In [26]:
calls['hour_of_day'] = (calls['timestamp'].dt.hour * 60 + calls['timestamp'].dt.minute ) / 60.

py.iplot(ff.create_distplot([calls['hour_of_day']],group_labels=["Hour"],bin_size=1, show_rug=False))

In [28]:
px.violin(calls.sort_values("CVDOW"), y="hour_of_day", x="Day", box=True, points="all", hover_name="CVLEGEND")

In [30]:
calls['OFFENSE'].value_counts().iplot(kind="bar")
calls['CVLEGEND'].value_counts().iplot(kind="bar")